In [5]:
import random
insignificant_tokens=[' ','!', '"','#','$','%','&','*','+','-','(',')',',','.','/','0','1','2','3','4','5','6','7','8',
                      '9',';','<','>','=','?','@','|','«','»','`','[',']',"'",'\\']
language_ids=['bg','bs','cz','es-AR','es-ES','hr','id','mk','my','pt-BR','pt-PT','sk','sr']
class Result:
    expected=""
    actual=""
class Language: #for each language in data set, we create a Language class
   
    def __init__ (self,lang_id):
        self.chars={} # storing characters and their counts
        self.prob={} #storing p(c|l) probability for each character
        # corresponding language id 
        self.lang_id=lang_id
    def calculate_probability(self,sentence):
        s=list(sentence)
        s=[x for x in s if x not in insignificant_tokens]
        total=0
        for ch in s:
            if self.chars.has_key(ch):
                total+=self.prob[ch]
            else: 
                return 0
        return total
    def get_chars(self):
        return self.chars
    def get_prob(self):
        return self.prob
    def get_lang_id(self):
        return self.lang_id
with open ("Corpus/Raw Corpus.txt") as f:
    corpus = f.readlines()
#print(len(corpus))
#idx=corpus[0].rfind("\t")
#print (corpus[0][0:idx])
#print (corpus[0][idx+1:len(corpus[0])

languages=[]
model=[]
for i in range(13): #divides data set as languages
    languages.append("")
    start=i*2000
    end=(i+1)*2000
    languages[i]=corpus[start:end] #2k sentence for each language

training_set=[]
test_set=[]
for idx,l in enumerate(languages):
    lang=Language(language_ids[idx])
    random.shuffle(l)
    
    
    training_partition=l[0:1800]
    training_set.extend(training_partition)
 
    training_partition=[i.split('\t')[0] for i in training_partition] #remove language identifier at the last of the sentences
    for sentence in training_partition:
        for letter in sentence:
            if letter not in insignificant_tokens:
                if lang.get_chars().has_key(letter):
                    nominator=lang.get_chars()[letter]
                else:
                    nominator=0
                nominator+=1
                lang.get_chars()[letter]=nominator
                
    test_partition=l[1800:2000]
    test_set.extend(test_partition)
    test_part=[i.split('\t')[0] for i in test_partition] #remove language identifier at the last of the sentences
    
    unk=list(''.join(set(''.join(test_part))))
    unk=[x for x in unk if x not in insignificant_tokens]
    unk=[x for x in unk if x not in lang.get_chars().keys()]
    print(unk)
    if len(unk)!=0: # add unknowns characters in test set for smoothing
        for l in unk:
            lang.get_chars()[l]=0
        
    for ch in lang.chars: #laplace smoothing
        lang.get_chars()[ch]+=1
    
    denominator=sum(lang.get_chars().values())+len(lang.get_chars().values()) #for performance
    for letter in lang.get_chars(): #calculate probabilities
        lang.get_prob()[letter]=(lang.get_chars()[letter]+1)/float(denominator)
  

    model.append(lang)


[]
['\x99']
[]
[]
[]
[]
[]
['X']
['\xad']
['\x84']
[]
[]
[]


In [6]:
i=0
for sentence in test_set:
    s=sentence.split('\t')
    probabilities={}
    
    for l_model in model:
        probabilities[l_model.get_lang_id()]=l_model.calculate_probability(s[0])
    if s[1].strip()==max(probabilities.items(), key=lambda k: k[1])[0]:
        i+=1
    
print (i)
print (len(test_set))
print((100.0*i)/float(len(test_set)))


1238
2600
47.6153846154


In [ ]:

def write_files(file_name,array):
    with open (file_name, mode='wt') as t_file:
        for item in array:
            t_file.write(item)
print(len(training_set))
print(len(test_set))
write_files("Training set.txt",training_set)
write_files("Test set.txt",test_set)